In [132]:
import cv2
import numpy as np
import tensorflow as tf
import dlib
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [152]:
face_classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
eyes_classifier = cv2.CascadeClassifier('./haarcascade_eye.xml')

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat')

model=load_model('../model/henry_model.h5')
model.summary()

def to_grayscale_then_rgb(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.grayscale_to_rgb(image)
    return image

close_frame = 0
prev_eyes = []
def face_ROI(frame):
    global close_frame, prev_eyes
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(frame_gray, 1.3, 3, minSize=(30, 30))
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        face_gray = frame_gray[y:y+h, x:x+w]
        face = frame[y:y+h, x:x+w]
#       cv2.imwrite("face.jpg", face)
        eyes = eyes_classifier.detectMultiScale(face_gray)
#         print("face height", str(h))
        new_eyes = []
        for (x2,y2,w2,h2) in eyes:
            
            if y2+h2/2 > h/2: # pass if the eye is at the bottom
                continue
            eye = face_gray[y2:y2+h2, x2:x2+w2]
            eye = cv2.cvtColor(eye, cv2.COLOR_GRAY2BGR)
            eye = cv2.resize(eye, (50, 50))
            eye = np.array(eye)
            eye = np.expand_dims(eye, axis=0)
            prev_eyes.append((x2,y2,w2,h2))
            new_eyes.append(eye)
            
            cv2.rectangle(face,(x2,y2),(x2+w2,y2+h2),(0,0,255),2)
        if not new_eyes:
            
            for (ox,oy,ow,oh) in prev_eyes:
                cv2.rectangle(face,(ox,oy),(ox+ow,oy+oh),(0,0,255),2)
            
            
            close_frame += 1
        else:
            prev_eyes.clear()
            for cur_eye in new_eyes:
                prediction = model.predict(cur_eye)
#                 print(prediction)
                if(prediction[0][0] == 1):
                    close_frame += 1
                    break
                else:
                    close_frame = 0
        for cur_eye in prev_eyes:
            prediction = model.predict(cur_eye)
#                print(prediction)
            if(prediction[0][0] == 1):
                close_frame += 1
                break
            else:
                close_frame = 0            
            
        if close_frame >= 15:
            cv2.putText(frame, 
                        "DROWSY!!!", 
                        (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, 
                        (0, 255, 255), 
                        2, 
                        cv2.LINE_4)    

    cv2.imshow('TEAM12', frame)

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 16)        448       
_________________________________________________________________
max_pooling2d_105 (MaxPoolin (None, 24, 24, 16)        0         
_________________________________________________________________
batch_normalization_93 (Batc (None, 24, 24, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 32)        4640      
_________________________________________________________________
max_pooling2d_106 (MaxPoolin (None, 11, 11, 32)        0         
_________________________________________________________________
batch_normalization_94 (Batc (None, 11, 11, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 64)        

In [153]:
vid = cv2.VideoCapture(0)
print(vid.get(cv2.CAP_PROP_FPS))
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame,(500,250))
    face_ROI(frame)
    # Display the resulting frame
#     cv2.imshow('TEAM12', frame)
      
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    # After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

5.0


In [31]:
img = cv2.imread("eye.jpg")
img = cv2.resize(img, (50, 50))
img = np.array(img)
img = np.expand_dims(img, axis=0)
# destroy all windows
prediction = model.predict(img)
print(prediction)

[[0.]]
